## Practica 2B (Ascensores)
##### Alberto García Doménech - Pablo Daurell Marina (Grupo 03)

In [2]:
# Restriccion: Una persona por ascensor
# estado_ini = ( (personas), (ascensores) )
estado_ini = (
    (1,5), (0,0)
)

# estado_goal = (personas)
estado_goal = (5,10)

# Acciones
# Llamar ascensor persona (mueve un ascensor vacio a una planta)
# Subir ascensor persona (sube una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar ascensor persona (baja una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)

In [3]:
from search import *

class Ascensores(Problem):
    
    def __init__(self, initial, goal):
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        personas = list(estado[0])
        ascensores = list(estado[1])
        accs = list()
        for i, persona in enumerate(personas):
            for j, ascensor in enumerate(ascensores):
                if persona != ascensor:
                    accs.append(("Llamar", i, j))            # Persona i llama a asccensor j
                if persona == ascensor and int(persona) > 0:
                    accs.append(("Bajar", i, j))             # Bajar persona i en ascensor j
                if persona == ascensor and int(persona) < 12:
                    accs.append(("Subir", i, j))             # Subir persona i en ascensor j
        return accs
    
    def result(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar":
            ascensores[a] = int(personas[p])
        elif accion[0] == "Bajar":
            ascensores[a] -= 1
            personas[p] -= 1
        elif accion[0] == "Subir":
            ascensores[a] += 1
            personas[p] += 1
        return (tuple(personas), tuple(ascensores))
    
    def goal_test(self, estado):
        return estado[0] == self.goal
    
    def coste_de_aplicar_accion(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar":
            coste = abs(personas[p] - ascensores[a])
        elif accion[0] == "Bajar":
            coste = 1
        elif accion[0] == "Subir":
            coste = 1
        return coste


In [4]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((1, 5), (0, 0))

In [5]:
p.goal

(5, 10)

In [6]:
p.actions(p.initial)

[('Llamar', 0, 0), ('Llamar', 0, 1), ('Llamar', 1, 0), ('Llamar', 1, 1)]

In [7]:
p.result(p.initial, ("Llamar", 0, 0))

((1, 5), (1, 0))

In [8]:
p.coste_de_aplicar_accion(p.initial, ("Llamar", 1, 0))

5

### Heurísticas

In [15]:
# Numero de personas en una planta incorrecta a la de destino 
def wrong_pos(node):
    res = 0
    goal = estado_goal
    for i in range(len(node.state[0])):
        if goal[i] != node.state[0][i]:
            res += 1
    return res

# Distancia de cada persona a su planta destino
def p_distance(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])
    return dist

### Búsqueda ciega

In [11]:
%%time
breadth_first_graph_search(p).solution()

Wall time: 284 ms


[('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0)]

In [12]:
%%time
depth_first_graph_search(p).solution()

Wall time: 46.9 ms


[('Llamar', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Llamar', 1, 0),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Llamar', 1, 0),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Llamar', 0, 1),
 ('Llamar', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 0, 1),
 ('Llamar', 1, 1),
 ('Subir', 1, 1),
 ('Llamar', 1, 0),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Bajar', 1, 1),
 ('Llamar', 1, 0),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 (

### Búsqueda heurística

In [13]:
%%time
astar_search(p, p_distance).solution()

Wall time: 15 ms


[('Llamar', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1)]

In [16]:
%%time
astar_search(p, wrong_pos).solution()

Wall time: 1.08 s


[('Llamar', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 0, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1),
 ('Subir', 1, 1)]